In [295]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, BaseCrossValidator
from sklearn.metrics import roc_auc_score
from sklearn.cluster import MiniBatchKMeans # for efficient clustering of environmental space

from IPython.display import display, HTML

from mycoCarte import Utils

In [296]:
# Split features
def split_features(df):
    feature_names = []
    categorical_feats = []
    numeric_feats = []

    for col in df.columns:
        feature_names.append(col)
        if col == 'FID':
            pass
        
        if df[col].dtype == 'object':
            categorical_feats.append(col) 
        else:
            numeric_feats.append(col)
    return feature_names, categorical_feats, numeric_feats

In [297]:
decimal_longitude = [-79.3439314990000071,-63.9999979090000011]
decimal_latitude = [45.0000682390000009, 50.0000022050000013]

In [298]:
df = pd.read_csv('/home/egodin/Documents/projects/mycoCarte/data/interim/model/preprocessedOccurencesData.csv', index_col=0 )
print(df.shape)
df.reset_index(drop= True, inplace= True)


(5833, 60)


In [299]:
df_grid = pd.read_csv('/home/egodin/Documents/projects/mycoCarte/data/interim/model/testData.csv', index_col= 0)

feature_names, categorical_feats, numeric_feats = split_features(df_grid)

df_grid = df_grid[numeric_feats]


In [300]:
# Keep Only EnvVars 
rows = ['FID', 'species','ty_couv_et','cl_dens','cl_haut','cl_age_et','etagement','cl_pent','hauteur','dep_sur','cl_drai','tree_diver','tree_shannon_index','regionCode','bioclim_01','bioclim_02','bioclim_03','bioclim_04','bioclim_05','bioclim_08','bioclim_09','bioclim_10','bioclim_12','bioclim_15','bioclim_16','bioclim_17','bioclim_18','populationDensity','distanceToUrban','distanceRo','urbanArea','airesProtegees','block_id']

df = df[rows]

# Encode bioclim to human-readable
df = Utils.encodeBioclim(df)

# Split features 
feature_names, categorical_feats, numeric_feats = split_features(df)
numeric_feats.append('species')

df = df[numeric_feats]

In [301]:
#Top specie to begin with 
species = df.groupby('species')['species'].count().sort_values(ascending=False)
specie = species.index[0]
print(specie)

# Create mask, split and label occurences 
mask = df['species'] == specie
df['presence'] = 0
df['presence'] = np.where(df['species'] == specie, 1, df['presence'])
df['presence'] = np.where(df['species'] != specie, 0, df['presence'])

df.drop('species', axis = 1, inplace=True)

display(df)

Amanita muscaria


,FID,cl_dens,cl_haut,cl_age_et,cl_pent,hauteur,cl_drai,tree_diver,tree_shannon_index,regionCode,...,Precip of Wettest Quarter,Precip of Driest Quarter,Precip of Warmest Quarter,populationDensity,distanceToUrban,distanceRo,urbanArea,airesProtegees,block_id,presence
0,638239,4.0,7.0,49.166667,2.0,24.666667,30.0,8.0,1.282727,3.0,...,295.0,215.0,291.0,986.11646,1964.834563,4167.482896,0.0,1.0,5,0
1,712576,4.0,7.0,84.000000,1.0,29.200000,40.0,3.0,0.738706,2.0,...,288.0,187.0,287.0,7.87286,2725.787129,2281.643037,0.0,1.0,5,0
2,574477,4.0,7.0,70.555556,1.0,23.111111,30.0,13.0,1.262714,3.0,...,276.0,198.0,270.0,14.66654,653.286725,1153.768290,1.0,0.0,5,0
3,278733,4.0,6.0,67.727273,1.0,17.727273,40.0,17.0,1.325918,10.0,...,264.0,187.0,263.0,1036.18530,3152.703636,327.367270,0.0,0.0,2,0
4,278733,4.0,6.0,67.727273,1.0,17.727273,40.0,17.0,1.325918,10.0,...,264.0,187.0,263.0,1036.18530,3152.703636,327.367270,0.0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,574477,4.0,7.0,70.555556,1.0,23.111111,30.0,13.0,1.262714,3.0,...,276.0,198.0,270.0,14.66654,653.286725,1153.768290,1.0,0.0,5,0
5829,524968,4.0,7.0,52.500000,1.0,21.333333,30.0,7.0,1.398735,10.0,...,287.0,195.0,287.0,5.65433,1071.271211,4429.277767,0.0,1.0,5,0
5830,524968,4.0,7.0,52.500000,1.0,21.333333,30.0,7.0,1.398735,10.0,...,287.0,195.0,287.0,5.65433,1071.271211,4429.277767,0.0,1.0,5,0
5831,524968,4.0,7.0,52.500000,1.0,21.333333,30.0,7.0,1.398735,10.0,...,287.0,195.0,287.0,5.65433,1071.271211,4429.277767,0.0,1.0,5,0


Pseudo absences Strategy A.1

Target-Group Background (TGB)

Instead of randomly sampling the entire study area, you use the occurrence records of other species that were collected with the same sampling effort or methodology as your focal species.

Pseudo absences Strategy A.2


Instead of random sampling in geographic space, you stratify your study area based on environmental variables. This means you divide the environmental space into bins or "strata" (e.g., high temperature/low precipitation, low temperature/high precipitation). You then sample an equal (or proportional) number of background points from each environmental stratum.

In [302]:
# c) Environmental Stratification (using K-means on the *entire study area's* environmental space)

df_grid = df_grid.sample(10000, random_state=42)
print(df_grid.shape)

kmeans = MiniBatchKMeans(n_clusters=10, random_state=42, n_init=10) # 10 environmental strata

kmeans.fit(df_grid.dropna()) # Fit on environmental values from the study area
study_area_strata = kmeans.predict(df_grid.dropna())
unique_strata, counts = np.unique(study_area_strata, return_counts=True)
strata_area_proportions = counts / len(study_area_strata)

df['env_stratum'] = np.random.randint(0, 10, len(df))

display(df)


(10000, 29)


,FID,cl_dens,cl_haut,cl_age_et,cl_pent,hauteur,cl_drai,tree_diver,tree_shannon_index,regionCode,...,Precip of Driest Quarter,Precip of Warmest Quarter,populationDensity,distanceToUrban,distanceRo,urbanArea,airesProtegees,block_id,presence,env_stratum
0,638239,4.0,7.0,49.166667,2.0,24.666667,30.0,8.0,1.282727,3.0,...,215.0,291.0,986.11646,1964.834563,4167.482896,0.0,1.0,5,0,2
1,712576,4.0,7.0,84.000000,1.0,29.200000,40.0,3.0,0.738706,2.0,...,187.0,287.0,7.87286,2725.787129,2281.643037,0.0,1.0,5,0,9
2,574477,4.0,7.0,70.555556,1.0,23.111111,30.0,13.0,1.262714,3.0,...,198.0,270.0,14.66654,653.286725,1153.768290,1.0,0.0,5,0,3
3,278733,4.0,6.0,67.727273,1.0,17.727273,40.0,17.0,1.325918,10.0,...,187.0,263.0,1036.18530,3152.703636,327.367270,0.0,0.0,2,0,1
4,278733,4.0,6.0,67.727273,1.0,17.727273,40.0,17.0,1.325918,10.0,...,187.0,263.0,1036.18530,3152.703636,327.367270,0.0,0.0,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,574477,4.0,7.0,70.555556,1.0,23.111111,30.0,13.0,1.262714,3.0,...,198.0,270.0,14.66654,653.286725,1153.768290,1.0,0.0,5,0,5
5829,524968,4.0,7.0,52.500000,1.0,21.333333,30.0,7.0,1.398735,10.0,...,195.0,287.0,5.65433,1071.271211,4429.277767,0.0,1.0,5,0,5
5830,524968,4.0,7.0,52.500000,1.0,21.333333,30.0,7.0,1.398735,10.0,...,195.0,287.0,5.65433,1071.271211,4429.277767,0.0,1.0,5,0,4
5831,524968,4.0,7.0,52.500000,1.0,21.333333,30.0,7.0,1.398735,10.0,...,195.0,287.0,5.65433,1071.271211,4429.277767,0.0,1.0,5,0,9


In [305]:
num_pseudo_absences = 10000 # Desired number of background points
pseudo_abs_points = []
for stratum_id in unique_strata:
    # Number of pseudo-absences to sample for this stratum based on its proportion of the study area
    n_sample_stratum = int(num_pseudo_absences * strata_area_proportions[np.where(unique_strata == stratum_id)[0][0]])
    
    # Get all target group points that fall into this environmental stratum
    tg_points_in_stratum = df[df['env_stratum'] == stratum_id]
    
    if not tg_points_in_stratum.empty:
        # Sample randomly from these TGB points (with replacement if needed)
        sample = tg_points_in_stratum.sample(n=min(n_sample_stratum, len(tg_points_in_stratum)),
                                              replace=n_sample_stratum > len(tg_points_in_stratum),
                                              random_state=42)
        pseudo_abs_points.append(sample)


if pseudo_abs_points:
    pseudo_absences_df = pd.concat(pseudo_abs_points).drop(columns=['env_stratum'])
    pseudo_absences_df['presence'] = 0 # Label as pseudo-absence
else:
    print("Warning: No background points generated. Check your stratification and TGB data.")
    # Fallback to random or just exit
    pseudo_absences_df = pd.DataFrame(columns=['latitude', 'longitude', 'species_presence'])

display(pseudo_absences_df)


,FID,cl_dens,cl_haut,cl_age_et,cl_pent,hauteur,cl_drai,tree_diver,tree_shannon_index,regionCode,...,Precip of Wettest Quarter,Precip of Driest Quarter,Precip of Warmest Quarter,populationDensity,distanceToUrban,distanceRo,urbanArea,airesProtegees,block_id,presence
1037,515526,4.0,6.0,79.500000,1.0,18.100000,30.0,14.0,1.450058,10.0,...,294.0,203.0,294.0,167.25156,54.982292,5090.273627,1.0,0.0,5,0
4588,864533,4.0,6.0,64.090909,1.0,17.000000,20.0,20.0,1.402854,1.0,...,350.0,197.0,350.0,17.27663,1487.699646,2168.316264,0.0,1.0,0,0
2844,624041,3.0,5.0,51.666667,1.0,16.000000,20.0,13.0,0.998683,3.0,...,275.0,195.0,275.0,1191.04456,432.556256,1605.503891,1.0,0.0,5,0
1054,392895,4.0,6.0,76.666667,2.0,20.000000,30.0,14.0,1.805604,10.0,...,296.0,198.0,289.0,2.97917,2131.266852,1668.097934,0.0,1.0,5,0
736,796043,4.0,6.0,78.571429,2.0,18.928571,30.0,16.0,1.526211,3.0,...,365.0,249.0,365.0,0.00000,932.955424,1631.557158,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4783,915305,3.0,5.0,36.000000,4.0,17.400000,20.0,8.0,1.184537,12.0,...,405.0,276.0,396.0,96.55774,959.487429,2313.142810,0.0,0.0,0,0
1070,1039981,3.0,6.0,70.000000,2.0,16.833333,30.0,8.0,0.893518,12.0,...,296.0,213.0,284.0,13.52508,1224.530890,3304.934053,1.0,0.0,0,0
1024,999854,4.0,6.0,36.363636,2.0,16.818182,30.0,13.0,1.216903,5.0,...,373.0,200.0,373.0,14.75888,1840.817577,1368.967739,0.0,0.0,0,0
5187,263705,4.0,5.0,60.000000,4.0,15.300000,20.0,11.0,1.185118,7.0,...,285.0,185.0,285.0,5.97321,1361.914422,3938.770270,0.0,0.0,2,0


In [304]:
full_data_df = pd.concat([df, pseudo_absences_df], ignore_index=True)


Pseudo absences Strategy B

Use random background points across the entire study area or a well-defined accessible area

Include the bias covariate as a predictor in your logistic regression model. 